# Semantic ID Recommender - Training Test

This notebook tests the full training pipeline in Google Colab.

**Requirements:**
- Google Colab with GPU runtime (T4 is sufficient)
- ~10 minutes for full test

**What this tests:**
1. RQ-VAE training (semantic ID learning)
2. Training data generation
3. LLM fine-tuning with Unsloth
4. Inference test

## Setup

In [ ]:
!nvidia-smi

In [ ]:
# Install uv
!pip install uv

# Clone private repo (skip if already exists)
import os
if not os.path.exists("semantic_id_recommender"):
    !git clone https://github.com/charleslow/semantic_id_recommender.git
else:
    print("✓ Repo already exists")
    
%cd semantic_id_recommender
!git pull --ff-only

In [ ]:
# Install dependencies (let uv resolve versions compatible with Colab's environment)
!uv pip install --system \
    torch torchvision torchaudio \
    transformers datasets accelerate \
    vector-quantize-pytorch sentence-transformers \
    lightning wandb omegaconf einops tqdm rich \
    pydantic unsloth psutil

# Fix protobuf version conflict
!uv pip install --system "protobuf>=3.20,<5"

# Note: We skip using uv.lock because Colab has pre-installed packages
# that may conflict with locked versions

## 1. Import from Package

In [ ]:
from unsloth import FastLanguageModel

In [ ]:
# Add repo root to Python path for imports
import sys
sys.path.insert(0, ".")

# Import from the cloned package
import torch
from src.rqvae.model import SemanticRQVAE, SemanticRQVAEConfig
from src.rqvae.trainer import RQVAETrainer
from src.rqvae.dataset import ItemEmbeddingDataset

print("✓ Package imports successful")

## 2. Inspect Dataset

In [ ]:
import json

items = []
with open("data/ag_news_500.jsonl") as f:
    for line in f:
        items.append(json.loads(line))

print(f"Loaded {len(items)} items")
print(f"\nKeys: {items[0].keys()}")
print(f"\nSample items:")
for item in items[:5]:
    print(f"  [{item['category']}] {item['item_id']}: {item['title'][:60]}...")

## 3. Generate Embeddings

In [ ]:
# Use the package's ItemEmbeddingDataset to generate embeddings
# Using GTE-Qwen2 - a powerful Qwen-based embedding model (1536 dim)
dataset = ItemEmbeddingDataset.from_catalogue(
    catalogue_path="data/ag_news_500.jsonl",
    embedding_model="Qwen/Qwen3-Embedding-0.6B",
    cache_path="data/embeddings_qwen3_0.6b.pt",  # New cache for different model
)

print(f"\nDataset info:")
print(f"  Number of items: {len(dataset)}")
print(f"  Embedding dim: {dataset.embeddings.shape[1]}")
print(f"  Item IDs (first 5): {dataset.item_ids[:5]}")

# Keep embeddings tensor for later use
embeddings = dataset.embeddings

In [ ]:
embeddings.shape

In [ ]:
import matplotlib.pyplot as plt
from collections import Counter

# Category distribution
categories = [item['category'] for item in items]
cat_counts = Counter(categories)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Bar chart of categories
ax1 = axes[0]
ax1.bar(cat_counts.keys(), cat_counts.values(), color=['#2ecc71', '#3498db', '#e74c3c', '#9b59b6'])
ax1.set_xlabel('Category')
ax1.set_ylabel('Count')
ax1.set_title('Distribution by Category')
for i, (cat, count) in enumerate(cat_counts.items()):
    ax1.text(i, count + 2, str(count), ha='center')

# Content length distribution
content_lengths = [len(item['content']) for item in items]
ax2 = axes[1]
ax2.hist(content_lengths, bins=30, color='#3498db', edgecolor='white')
ax2.set_xlabel('Content Length (chars)')
ax2.set_ylabel('Count')
ax2.set_title('Distribution of Article Lengths')
ax2.axvline(sum(content_lengths)/len(content_lengths), color='red', linestyle='--', label=f'Mean: {sum(content_lengths)//len(content_lengths)}')
ax2.legend()

plt.tight_layout()
plt.show()

print(f"\nSummary:")
print(f"  Total articles: {len(items)}")
print(f"  Categories: {dict(cat_counts)}")
print(f"  Avg content length: {sum(content_lengths)//len(content_lengths)} chars")

## 4. Train RQ-VAE

In [ ]:
import lightning as L
from torch.utils.data import DataLoader, random_split
from lightning.pytorch.loggers import WandbLogger
import wandb

# Initialize wandb for the entire experiment
wandb.init(
    project="semantic-id-recommender",
    name="full-pipeline-test",
    config={
        "embedding_model": "Qwen/Qwen3-Embedding-0.6B",
        "num_items": len(items),
    }
)

config = SemanticRQVAEConfig(
    embedding_dim=1024,  # Match embedding model output dimension
    hidden_dim=256,      # Larger hidden dim for bigger embeddings
    codebook_size=32,
    num_quantizers=3,
    threshold_ema_dead_code=1,
)

# Log RQ-VAE config to wandb
wandb.config.update({
    "rqvae_embedding_dim": config.embedding_dim,
    "rqvae_hidden_dim": config.hidden_dim,
    "rqvae_codebook_size": config.codebook_size,
    "rqvae_num_quantizers": config.num_quantizers,
})

trainer_module = RQVAETrainer(config=config, learning_rate=1e-3)

# Split dataset into train/val (90/10)
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
print(f"Train: {len(train_dataset)}, Val: {len(val_dataset)}")

# Create WandbLogger for Lightning
wandb_logger = WandbLogger(experiment=wandb.run)

trainer = L.Trainer(
    max_epochs=10,
    accelerator="auto",
    devices=1,
    enable_progress_bar=True,
    log_every_n_steps=1,
    logger=wandb_logger,  # Log to wandb
)
trainer.fit(trainer_module, train_loader, val_loader)
print("✓ RQ-VAE training complete")

In [ ]:
# Check training diagnostics
print("=== Training Diagnostics ===\n")

# Get logged metrics from trainer
logged_metrics = trainer.logged_metrics
print("Final logged metrics:")
for key, value in sorted(logged_metrics.items()):
    if isinstance(value, float):
        print(f"  {key}: {value:.4f}")
    else:
        print(f"  {key}: {value}")

# Compute codebook stats on full dataset
model = trainer_module.model
model.eval()
device = next(model.parameters()).device

with torch.no_grad():
    all_indices = model.get_semantic_ids(embeddings.to(device))
    stats = model.compute_codebook_stats(all_indices)

print(f"\n=== Codebook Statistics (full dataset) ===")
print(f"Average perplexity: {stats['avg_perplexity']:.2f} / {config.codebook_size} (max)")
print(f"Average usage: {stats['avg_usage']*100:.1f}%")
print(f"\nPer-level breakdown:")
for q in range(config.num_quantizers):
    perp = stats['perplexity_per_level'][q].item()
    usage = stats['usage_per_level'][q].item() * 100
    print(f"  Level {q}: perplexity={perp:.1f}, usage={usage:.1f}%")

# Check for semantic ID collisions
unique_ids = len(set(model.semantic_id_to_string(all_indices)))
print(f"\n=== Semantic ID Quality ===")
print(f"Total items: {len(embeddings)}")
print(f"Unique semantic IDs: {unique_ids}")
print(f"Collision rate: {(1 - unique_ids/len(embeddings))*100:.1f}%")

## 5. Generate Semantic IDs

In [ ]:
# Generate semantic IDs for all items using the trained model
model = trainer_module.model
model.eval()

device = next(model.parameters()).device
with torch.no_grad():
    indices = model.get_semantic_ids(embeddings.to(device))
    semantic_strings = model.semantic_id_to_string(indices)

# Create mapping
item_to_semantic = {}
semantic_to_item = {}

for i, item in enumerate(items):
    item_id = item["item_id"]  # AG News uses item_id
    sem_id = semantic_strings[i]
    item_to_semantic[item_id] = {
        "codes": indices[i].cpu().tolist(),
        "semantic_id": sem_id,
    }
    semantic_to_item[sem_id] = item_id

# Save mapping
mapping = {
    "item_to_semantic": item_to_semantic,
    "semantic_to_item": semantic_to_item,
    "config": {"num_quantizers": config.num_quantizers, "codebook_size": config.codebook_size}
}

with open("data/semantic_ids.json", "w") as f:
    json.dump(mapping, f, indent=2)

# Show examples
print("\nExample semantic IDs:")
for i in range(5):
    print(f"  {items[i]['title'][:30]:30} -> {semantic_strings[i]}")

print(f"\n✓ Generated {len(semantic_to_item)} unique semantic IDs")

In [ ]:
# Examine embedding similarity vs semantic ID similarity
import torch.nn.functional as F

# Compute pairwise cosine similarities
normed = F.normalize(embeddings, dim=1)
similarities = normed @ normed.T

# Mask out diagonal and near-duplicates (sim > 0.99)
similarities_filtered = similarities.clone()
similarities_filtered.fill_diagonal_(-1)
mask = similarities_filtered > 0.99
similarities_filtered[mask] = -1  # Exclude near-duplicates

# Find most similar pair (excluding duplicates)
max_idx = similarities_filtered.argmax()
i_close, j_close = max_idx // len(items), max_idx % len(items)

# Find most dissimilar pair
similarities_for_min = similarities.clone()
similarities_for_min.fill_diagonal_(2)  # Exclude self
min_idx = similarities_for_min.argmin()
i_far, j_far = min_idx // len(items), min_idx % len(items)

print("=" * 70)
print("CLOSEST PAIR (high embedding similarity, excluding duplicates)")
print("=" * 70)
print(f"Cosine similarity: {similarities[i_close, j_close]:.4f}")
print(f"\nItem A [{items[i_close]['category']}]:")
print(f"  Title: {items[i_close]['title'][:80]}")
print(f"  Semantic ID: {semantic_strings[i_close]}")
print(f"\nItem B [{items[j_close]['category']}]:")
print(f"  Title: {items[j_close]['title'][:80]}")
print(f"  Semantic ID: {semantic_strings[j_close]}")

# Check how many codes match
codes_a = indices[i_close].tolist()
codes_b = indices[j_close].tolist()
matching = sum(a == b for a, b in zip(codes_a, codes_b))
print(f"\nCodes A: {codes_a}")
print(f"Codes B: {codes_b}")
print(f"Matching codes: {matching}/{len(codes_a)} ✓" if matching > 0 else f"Matching codes: {matching}/{len(codes_a)}")

print("\n" + "=" * 70)
print("FARTHEST PAIR (low embedding similarity)")
print("=" * 70)
print(f"Cosine similarity: {similarities[i_far, j_far]:.4f}")
print(f"\nItem A [{items[i_far]['category']}]:")
print(f"  Title: {items[i_far]['title'][:80]}")
print(f"  Semantic ID: {semantic_strings[i_far]}")
print(f"\nItem B [{items[j_far]['category']}]:")
print(f"  Title: {items[j_far]['title'][:80]}")
print(f"  Semantic ID: {semantic_strings[j_far]}")

codes_a = indices[i_far].tolist()
codes_b = indices[j_far].tolist()
matching = sum(a == b for a, b in zip(codes_a, codes_b))
print(f"\nCodes A: {codes_a}")
print(f"Codes B: {codes_b}")
print(f"Matching codes: {matching}/{len(codes_a)}")

# Check for duplicates in dataset
dup_count = (similarities > 0.999).sum().item() - len(items)  # Subtract diagonal
print(f"\n⚠️  Found {dup_count//2} duplicate pairs in dataset (cosine sim > 0.999)")

## 6. Generate Training Data

In [ ]:
# Generate training data using the package utilities
from src.llm.data import format_for_chat

# Create items with semantic IDs for training
items_with_sem_ids = []
for item in items:
    item_copy = item.copy()
    item_copy["semantic_id"] = item_to_semantic[item["item_id"]]["semantic_id"]
    items_with_sem_ids.append(item_copy)

# Generate training examples
from src.llm.data import generate_training_examples
raw_examples = generate_training_examples(
    items_with_sem_ids, 
    num_examples_per_item=10,
    query_templates=[
        # Direct title queries
        "{title}",
        "Find me news about {title}",
        "I'm looking for articles on {title}",
        "Show me {title}",
        "Search for {title}",
        # Category-based queries
        "News about {category}: {title}",
        "{category} news: {title}",
        "Latest {category} article about {title}",
        # Question formats
        "What's the news about {title}?",
        "Any articles on {title}?",
        "Do you have news about {title}?",
        # Recommendation style
        "Recommend articles about {title}",
        "Suggest news on {title}",
        # Short/casual queries
        "news {title}",
        "article {title}",
        "{category} {title}",
    ],
)

# Format for chat and split
import random
formatted_examples = format_for_chat(raw_examples)
random.shuffle(formatted_examples)
split_idx = int(len(formatted_examples) * 0.9)
train_examples = formatted_examples[:split_idx]
val_examples = formatted_examples[split_idx:]

print(f"✓ Generated {len(train_examples)} train, {len(val_examples)} val examples")
print(f"\nExample:")
print(f"  Query: {train_examples[0]['messages'][1]['content'][:50]}...")
print(f"  Target: {train_examples[0]['messages'][2]['content']}")

## 7. Fine-tune LLM with Unsloth

In [ ]:
# Update wandb config for LLM training (wandb already initialized)
wandb.config.update({
    "base_model": "unsloth/SmolLM2-135M-Instruct",
    "num_train_examples": len(train_examples),
})

In [ ]:
from unsloth import FastLanguageModel
from datasets import Dataset

# Use SmolLM - very small model (135M parameters)
BASE_MODEL = "unsloth/SmolLM2-135M-Instruct"
MAX_SEQ_LENGTH = 256

print(f"Loading {BASE_MODEL}...")
llm_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=BASE_MODEL,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=None,
    load_in_4bit=True,
)

print("✓ Model loaded")

In [ ]:
# Add semantic ID tokens using package utility
from src.llm.finetune import add_semantic_tokens

tokenizer = add_semantic_tokens(tokenizer, config.num_quantizers, config.codebook_size)
llm_model.resize_token_embeddings(len(tokenizer))

print(f"✓ Added {config.num_quantizers * config.codebook_size} semantic tokens")

In [ ]:
# Add LoRA adapters
llm_model = FastLanguageModel.get_peft_model(
    llm_model,
    r=32,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42,
)

print("✓ LoRA adapters added")

In [ ]:
# Prepare dataset using package formatting function
def format_single_example(messages):
    """Format messages into prompt string."""
    text = ""
    for msg in messages:
        role, content = msg["role"], msg["content"]
        if role == "system":
            text += f"<|system|>\n{content}\n"
        elif role == "user":
            text += f"<|user|>\n{content}\n"
        elif role == "assistant":
            text += f"<|assistant|>\n{content}\n"
    return text

train_texts = [{"text": format_single_example(ex["messages"])} for ex in train_examples]
val_texts = [{"text": format_single_example(ex["messages"])} for ex in val_examples]

train_dataset = Dataset.from_list(train_texts)
val_dataset = Dataset.from_list(val_texts)

print(f"✓ Dataset prepared: {len(train_dataset)} train, {len(val_dataset)} val")
print(f"Sample: {train_texts[0]['text'][:100]}...")

In [ ]:
# Fix: Unsloth's compiled cache needs psutil in builtins
import psutil
import builtins
builtins.psutil = psutil

from trl import SFTTrainer
from transformers import TrainingArguments

# Training arguments
training_args = TrainingArguments(
    output_dir="checkpoints/llm",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    warmup_ratio=0.03,
    logging_steps=10,
    weight_decay=0.01,
    max_grad_norm=1.0,
    save_strategy="no",
    eval_strategy="steps",      # Enable evaluation
    eval_steps=50,              # Evaluate every 50 steps
    fp16=True,
    optim="adamw_8bit",
    report_to="wandb",
)

trainer = SFTTrainer(
    model=llm_model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,   # Add validation dataset
    args=training_args,
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LENGTH,
    packing=False,
)

print("✓ Trainer configured")

In [ ]:
# Train!
print("Starting training...")
trainer.train()
print("✓ Training complete!")

## 8. Test Inference

In [ ]:
"""
LLM fine-tuning script using Unsloth.

Fine-tunes a base model to generate semantic IDs from user queries.
"""

from pathlib import Path

from datasets import Dataset
from trl import SFTTrainer
from transformers import TrainingArguments

from .data import get_semantic_id_tokens


def add_semantic_tokens(tokenizer, num_quantizers: int = 4, codebook_size: int = 256):
    """
    Add semantic ID special tokens to tokenizer.

    Args:
        tokenizer: HuggingFace tokenizer
        num_quantizers: Number of RQ-VAE quantizers
        codebook_size: Size of each codebook

    Returns:
        Updated tokenizer
    """
    special_tokens = get_semantic_id_tokens(num_quantizers, codebook_size)
    tokenizer.add_special_tokens({"additional_special_tokens": special_tokens})
    return tokenizer


def formatting_prompts_func(examples):
    """Format examples for training using chat template."""
    output_texts = []
    for messages in examples["messages"]:
        text = ""
        for msg in messages:
            role = msg["role"]
            content = msg["content"]
            if role == "system":
                text += f"<|system|>\n{content}\n"
            elif role == "user":
                text += f"<|user|>\n{content}\n"
            elif role == "assistant":
                text += f"<|assistant|>\n{content}\n"
        output_texts.append(text)
    return output_texts


def finetune_model(
    train_dataset: Dataset,
    val_dataset: Dataset | None = None,
    base_model: str = "unsloth/Qwen3-4B",
    output_dir: str = "checkpoints/llm",
    num_quantizers: int = 4,
    codebook_size: int = 256,
    max_seq_length: int = 512,
    lora_r: int = 16,
    lora_alpha: int = 32,
    lora_dropout: float = 0.05,
    learning_rate: float = 2e-4,
    batch_size: int = 4,
    gradient_accumulation_steps: int = 4,
    max_epochs: int = 3,
    warmup_ratio: float = 0.03,
    push_to_hub: bool = False,
    hub_repo: str | None = None,
):
    """
    Fine-tune LLM using Unsloth for semantic ID generation.

    Args:
        train_dataset: Training dataset
        val_dataset: Validation dataset (optional)
        base_model: Base model name (Unsloth-compatible)
        output_dir: Directory to save checkpoints
        num_quantizers: Number of semantic ID quantizers
        codebook_size: Size of each codebook
        max_seq_length: Maximum sequence length
        lora_r: LoRA rank
        lora_alpha: LoRA alpha
        lora_dropout: LoRA dropout
        learning_rate: Learning rate
        batch_size: Batch size per device
        gradient_accumulation_steps: Gradient accumulation steps
        max_epochs: Number of training epochs
        warmup_ratio: Warmup ratio
        push_to_hub: Whether to push to HuggingFace Hub
        hub_repo: Hub repository name

    Returns:
        Trained model and tokenizer
    """
    from unsloth import FastLanguageModel

    # Load model with Unsloth
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=base_model,
        max_seq_length=max_seq_length,
        dtype=None,  # Auto-detect
        load_in_4bit=True,  # QLoRA
    )

    # Add semantic ID tokens
    tokenizer = add_semantic_tokens(tokenizer, num_quantizers, codebook_size)
    model.resize_token_embeddings(len(tokenizer))

    # Add LoRA adapters
    model = FastLanguageModel.get_peft_model(
        model,
        r=lora_r,
        lora_alpha=lora_alpha,
        lora_dropout=lora_dropout,
        target_modules=[
            "q_proj", "k_proj", "v_proj", "o_proj",
            "gate_proj", "up_proj", "down_proj",
        ],
        bias="none",
        use_gradient_checkpointing="unsloth",  # Memory efficient
        random_state=42,
    )

    # Training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=max_epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        learning_rate=learning_rate,
        warmup_ratio=warmup_ratio,
        logging_steps=10,
        save_strategy="epoch",
        eval_strategy="epoch" if val_dataset else "no",
        fp16=True,
        optim="adamw_8bit",
        report_to="wandb",
        push_to_hub=push_to_hub,
        hub_model_id=hub_repo,
    )

    # Create trainer
    trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        args=training_args,
        formatting_func=formatting_prompts_func,
        max_seq_length=max_seq_length,
        packing=False,
    )

    # Train
    trainer.train()

    # Save
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

    # Push to hub if configured
    if push_to_hub and hub_repo:
        model.push_to_hub(hub_repo)
        tokenizer.push_to_hub(hub_repo)

    return model, tokenizer


def load_finetuned_model(
    model_path: str,
    max_seq_length: int = 512,
    load_in_4bit: bool = True,
):
    """
    Load fine-tuned model for inference.

    Args:
        model_path: Path to saved model or HuggingFace repo
        max_seq_length: Maximum sequence length
        load_in_4bit: Whether to load in 4-bit

    Returns:
        Model and tokenizer
    """
    from unsloth import FastLanguageModel

    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=model_path,
        max_seq_length=max_seq_length,
        dtype=None,
        load_in_4bit=load_in_4bit,
    )

    # Set to inference mode
    FastLanguageModel.for_inference(model)

    return model, tokenizer


class SemanticIDGenerator:
    """Generator for semantic IDs with optional constrained decoding."""

    def __init__(
        self,
        model,
        tokenizer,
        num_quantizers: int = 4,
        codebook_size: int = 256,
        use_constrained_decoding: bool = True,
    ):
        """
        Initialize the generator.

        Args:
            model: Fine-tuned model
            tokenizer: Tokenizer with semantic ID tokens
            num_quantizers: Number of RQ-VAE quantizers
            codebook_size: Size of each codebook
            use_constrained_decoding: Whether to use constrained decoding
        """
        self.model = model
        self.tokenizer = tokenizer
        self.num_quantizers = num_quantizers
        self.codebook_size = codebook_size
        self.use_constrained_decoding = use_constrained_decoding

        # Build semantic token ID lookup
        self.semantic_token_ids = [
            tokenizer.convert_tokens_to_ids(f"[SEM_{q}_{c}]")
            for q in range(num_quantizers)
            for c in range(codebook_size)
        ]
        self.semantic_token_ids_set = set(self.semantic_token_ids)
        self.eos_token_id = tokenizer.eos_token_id

        self.system_prompt = (
            "You are a recommendation system. Given a user query, "
            "output the semantic ID of the most relevant item. "
            "Respond only with the semantic ID tokens."
        )

    def _get_allowed_tokens_for_position(self, position: int) -> list[int]:
        """Return allowed token IDs for each position in the semantic ID."""
        if position < self.num_quantizers:
            # Only allow tokens from the correct quantizer level
            start_idx = position * self.codebook_size
            end_idx = start_idx + self.codebook_size
            return self.semantic_token_ids[start_idx:end_idx]
        else:
            # After all quantizers, only allow EOS
            return [self.eos_token_id]

    def _prefix_allowed_tokens_fn(self, batch_id: int, input_ids) -> list[int]:
        """Constrained decoding: only allow valid semantic ID tokens in sequence."""
        generated = input_ids.tolist()
        sem_count = sum(1 for tok_id in generated if tok_id in self.semantic_token_ids_set)
        return self._get_allowed_tokens_for_position(sem_count)

    def _extract_semantic_id(self, generated_text: str) -> str:
        """Extract semantic ID from generated text."""
        if "<|assistant|>" in generated_text:
            semantic_id = generated_text.split("<|assistant|>")[-1].strip()
        else:
            semantic_id = generated_text.strip()

        # Clean up EOS token if present
        if self.tokenizer.eos_token:
            semantic_id = semantic_id.replace(self.tokenizer.eos_token, "").strip()

        return semantic_id

    def generate(
        self,
        query: str,
        temperature: float = 0.1,
    ) -> str:
        """
        Generate semantic ID for a query.

        Args:
            query: User query
            temperature: Sampling temperature (ignored if constrained)

        Returns:
            Generated semantic ID string
        """
        prompt = f"<|system|>\n{self.system_prompt}\n<|user|>\n{query}\n<|assistant|>\n"
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)

        if self.use_constrained_decoding:
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=self.num_quantizers + 1,
                do_sample=False,  # Greedy for constrained
                pad_token_id=self.tokenizer.pad_token_id,
                prefix_allowed_tokens_fn=self._prefix_allowed_tokens_fn,
            )
        else:
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=32,
                temperature=temperature,
                do_sample=temperature > 0,
                pad_token_id=self.tokenizer.pad_token_id,
            )

        generated = self.tokenizer.decode(outputs[0], skip_special_tokens=False)
        return self._extract_semantic_id(generated)

    def generate_beam(
        self,
        query: str,
        num_beams: int = 10,
        num_return_sequences: int = 10,
    ) -> list[tuple[str, float]]:
        """
        Generate multiple semantic IDs using beam search.

        Args:
            query: User query
            num_beams: Number of beams for beam search
            num_return_sequences: Number of sequences to return

        Returns:
            List of (semantic_id, score) tuples, sorted by score (highest first)
        """
        prompt = f"<|system|>\n{self.system_prompt}\n<|user|>\n{query}\n<|assistant|>\n"
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)

        outputs = self.model.generate(
            **inputs,
            max_new_tokens=self.num_quantizers + 1,
            num_beams=num_beams,
            num_return_sequences=min(num_return_sequences, num_beams),
            do_sample=False,
            pad_token_id=self.tokenizer.pad_token_id,
            prefix_allowed_tokens_fn=self._prefix_allowed_tokens_fn if self.use_constrained_decoding else None,
            output_scores=True,
            return_dict_in_generate=True,
            use_cache=False,  # Disable KV cache to fix beam search with LoRA/Unsloth
        )

        results = []
        sequences = outputs.sequences
        # Compute sequence scores (sum of log probs)
        scores = outputs.sequences_scores if hasattr(outputs, 'sequences_scores') else [0.0] * len(sequences)

        for seq, score in zip(sequences, scores):
            generated = self.tokenizer.decode(seq, skip_special_tokens=False)
            semantic_id = self._extract_semantic_id(generated)
            score_val = score.item() if hasattr(score, 'item') else float(score)
            results.append((semantic_id, score_val))

        # Sort by score (highest first) and deduplicate
        seen = set()
        unique_results = []
        for sem_id, score in sorted(results, key=lambda x: x[1], reverse=True):
            if sem_id not in seen:
                seen.add(sem_id)
                unique_results.append((sem_id, score))

        return unique_results

    def __call__(self, query: str, **kwargs) -> str:
        """Shorthand for generate()."""
        return self.generate(query, **kwargs)


def generate_semantic_id(
    model,
    tokenizer,
    query: str,
    max_new_tokens: int = 32,
    temperature: float = 0.1,
) -> str:
    """
    Generate semantic ID for a query (simple version without constrained decoding).

    For constrained decoding, use SemanticIDGenerator instead.

    Args:
        model: Fine-tuned model
        tokenizer: Tokenizer
        query: User query
        max_new_tokens: Maximum tokens to generate
        temperature: Sampling temperature

    Returns:
        Generated semantic ID string
    """
    system_prompt = (
        "You are a recommendation system. Given a user query, "
        "output the semantic ID of the most relevant item. "
        "Respond only with the semantic ID tokens."
    )

    prompt = f"<|system|>\n{system_prompt}\n<|user|>\n{query}\n<|assistant|>\n"

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=temperature > 0,
        pad_token_id=tokenizer.pad_token_id,
    )

    # Decode and extract semantic ID
    generated = tokenizer.decode(outputs[0], skip_special_tokens=False)
    # Extract part after <|assistant|>
    if "<|assistant|>" in generated:
        semantic_id = generated.split("<|assistant|>")[-1].strip()
    else:
        semantic_id = generated[len(prompt):].strip()

    return semantic_id


In [ ]:
# Set to inference mode and create generator with constrained decoding
FastLanguageModel.for_inference(llm_model)

from src.llm.finetune import SemanticIDGenerator

generator = SemanticIDGenerator(
    model=llm_model,
    tokenizer=tokenizer,
    num_quantizers=config.num_quantizers,
    codebook_size=config.codebook_size,
    use_constrained_decoding=True,
)

print("✓ Inference mode ready with constrained decoding")
print(f"  Semantic tokens: {config.num_quantizers * config.codebook_size} total")
print(f"  Output format: {config.num_quantizers} tokens (one per quantizer level)")

In [ ]:
# Test queries with beam search - show top 10 candidates and valid matches
test_queries = [
    "News about stock market and business",
    "Sports football game results",
    "Technology and science discoveries",
    "World news international politics",
]

NUM_BEAMS = 10

print("\n" + "="*70)
print("INFERENCE TESTS (Beam Search)")
print("="*70)

results_table = []
for query in test_queries:
    print(f"\n{'─'*70}")
    print(f"Query: {query}")
    print(f"{'─'*70}")
    
    # Get top candidates via beam search
    candidates = generator.generate_beam(query, num_beams=NUM_BEAMS, num_return_sequences=NUM_BEAMS)
    
    print(f"\nTop {len(candidates)} candidates:")
    valid_count = 0
    for rank, (sem_id, score) in enumerate(candidates, 1):
        if sem_id in semantic_to_item:
            item_id = semantic_to_item[sem_id]
            item = next(i for i in items if i["item_id"] == item_id)
            print(f"  {rank}. ✓ {sem_id} (score: {score:.2f})")
            print(f"     [{item['category']}] {item['title'][:55]}...")
            valid_count += 1
            results_table.append([query, rank, sem_id, score, item['category'], item['title'][:50], True])
        else:
            print(f"  {rank}. ✗ {sem_id} (score: {score:.2f}) - not in catalogue")
            results_table.append([query, rank, sem_id, score, "", "", False])
    
    print(f"\n  Valid matches: {valid_count}/{len(candidates)}")

# Log results to wandb
wandb.log({
    "inference_results": wandb.Table(
        columns=["query", "rank", "semantic_id", "score", "category", "title", "valid"],
        data=results_table
    )
})

print("\n" + "="*70)
print("✓ All tests complete!")
print("="*70)

# Finish wandb run
wandb.finish()

## Summary

This notebook tested:

1. **RQ-VAE Training** - ✓ Model learns to encode items as discrete codes
2. **Semantic ID Generation** - ✓ Each item gets a unique semantic ID
3. **Training Data Creation** - ✓ Query → Semantic ID pairs generated
4. **LLM Fine-tuning** - ✓ Model learns to predict semantic IDs
5. **Inference** - ✓ Model generates semantic IDs for new queries

### Next Steps

For production use:
- Use larger model (Qwen3-4B or Ministral-3B)
- Train for more epochs (3-5)
- Use full catalogue
- Deploy to Modal for serverless inference